In [51]:
%%writefile Dockerfile2
# ffmpeg build - simply change versions to compile
# This Dockerfile is based on docker image  rottenberg/ffmpeg
# The purpose behind this is to build ffmpeg inside a debian container.
# 	Then the next step is to export the compiled binaries into an untouched base debian image
# 	This results in a ~50% images size reduction.
# 	FYI:  this image is useable....I just like having a smaller base image to download.
# Some of these options I don't use, so I commented them out.
# My builds include only FFMPEG + libfdk_aac + latest x264 + Decklink(Blackmagic)
# I am including Decklink(Blackmagic) so I can utilize those devices
# I don't need anything else.  If you need anything included, email me and I can make alternative builds.
# I will be tracking the 'stable' rolling release of Debian

FROM		ubuntu:18.04
MAINTAINER	jack northrup <madhatterstamps@gmail.com>

ENV	FFMPEG_VERSION		3.2.4
	# monitor releases at https://github.com/FFmpeg/FFmpeg/releases
ENV	YASM_VERSION    	1.3.0
	# monitor releases at https://github.com/yasm/yasm/releases
ENV	FDKAAC_VERSION  	0.1.5
	# monitor releases at https://github.com/mstorsjo/fdk-aac/releases
#ENV	x264
	# this project does not use release versions at this time
	# monitor project at http://git.videolan.org/?p=x264.git;a=shortlog
#ENV	LAME_VERSION    	3.99.5
#ENV	FAAC_VERSION    	1.28
#ENV	XVID_VERSION    	1.3.3
#ENV	MPLAYER_VERSION 	1.1.1

ENV	SRC             	/usr/local
ENV	LD_LIBRARY_PATH 	${SRC}/lib
ENV	PKG_CONFIG_PATH 	${SRC}/lib/pkgconfig
RUN bash -c 'set -euo pipefail'

RUN DEBIAN_FRONTEND=noninteractive apt-get -y update && apt-get install -yq \
autoconf automake gcc build-essential git libtool make nasm zlib1g-dev tar curl apt-utils 
# YASM
RUN DEBIAN_FRONTEND=noninteractive apt-get install -y x11vnc xvfb xterm fluxbox \
frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r \
libass-dev libdc1394-22-dev libmp3lame-dev libmp3lame0 libsox-fmt-mp3 libtheora-dev \
libtheora-bin libtheora0 libvorbis-dev libvpx-dev libwavpack-dev libwavpack1 \
libx264-152 libx264-dev libx265-146 libx265-dev x265 libx265-146 libx265-dev x265 \
libxvidcore-dev libxvidcore4 libopenjp2-7-dev libopenjp2-7 libopus-dev librtmp-dev librtmp1 \
libsoxr-dev libsoxr-lsr0 libsoxr0 speex libspeex-dev libvo-amrwbenc-dev libwebp-dev libwebp6 \
libespeak-dev libespeak1 python3-pyopencl libclc-dev pulseaudio yasm
RUN apt install -y \
youtube-dl winff libavcodec-extra libavcodec-extra57 libvo-aacenc-dev vlc-plugin-access-extra \
meld rubberband-ladspa libiec61883-dev ladspa-sdk libopencv-apps-dev libopencv-apps0d \
libopencv-calib3d-dev libopencv-calib3d3.2 libopencv-contrib-dev libopencv-contrib3.2 \
libopencv-core-dev libopencv-core3.2 libopencv-dev libopencv-features2d-dev \
libopencv-features2d3.2 libopencv-flann-dev libopencv-flann3.2 libopencv-highgui-dev \
libopencv-highgui3.2 libopencv-imgcodecs-dev libopencv-imgcodecs3.2 libopencv-imgproc-dev 
RUN apt install -y \
libopencv-imgproc3.2 libopencv-ml-dev libopencv-ml3.2 libopencv-objdetect-dev libopencv-objdetect3.2 libopencv-photo-dev \
libopencv-photo3.2 libopencv-shape-dev libopencv-shape3.2 libopencv-stitching-dev libopencv-stitching3.2 libopencv-superres-dev \
libopencv-superres3.2 libopencv-ts-dev libopencv-video-dev libopencv-video3.2 libopencv-videoio-dev libopencv-videoio3.2 libopencv-videostab-dev \
libopencv-viz-dev libopencv-viz3.2 libchromaprint-dev gnutls-bin libgnutls-openssl27 libgnutls28-dev \
libgnutls30 libnettle6 nettle-dev \
libhogweed4 locate nano 

COPY ffmpeg /home/ffmpeg
WORKDIR /home/ffmpeg
RUN apt install -y libgme-dev libgme0 libgmenuharness-dev libbluray-bin libbluray-dev libbluray2 \
libgsm-tools libgsm0710-0 libgsm0710-dev libgsm0710mux3 libgsm1 libgsm1-dbg libgsm1-dev libgsmme-dev \
libdrm-amdgpu1 libdrm-common libdrm-dev caca-utils libcaca-dev libcaca0 libcacard-dev libcacard-tools \
libcacard0 libbs2b-dev libbs2b0 libgsm-tools libgsm0710-0 libgsm0710-dev libgsm0710mux3 libgsm1-dbg \
libgsm1-dev libgsmme-dev libmysofa-dev libmysofa-utils libmysofa0 libopencore-amrwb0-dbg \
libopencore-amrnb-dev libopencore-amrnb0 libopencore-amrnb0-dbg libopencore-amrwb-dev libopencore-amrwb0 \
libopenmpt-dev libopenmpt-modplug-dev libopenmpt-modplug1 libopenmpt0 openmpt123 \
librubberband-dev librubberband2 rubberband-cli rubberband-ladspa libpulse0 libpulse-dev \
libshine-dev libshine3 liquidsoap-plugin-shine shineenc libsnappy-dev libsnappy1v5 \
libssh-4 libssh-dev libavfilter-extra6 libtesseract-dev libtesseract4 tesseract-ocr-eng \
libsox-fmt-mp3 libtwolame-dev libtwolame0 libx265-146 libx265-dev x265 \
libopenal-dev libopenal1 libcdio-dev libcdio-paranoia-dev libqt4-opengl libqt4-opengl-dev

RUN PATH="$HOME/bin:$PATH" PKG_CONFIG_PATH="$HOME/ffmpeg_build/lib/pkgconfig" ./configure \
 --prefix="$HOME/ffmpeg_build" \
 --pkg-config-flags="--static" \
 --extra-cflags="-I$HOME/ffmpeg_build/include" \
 --extra-ldflags="-L$HOME/ffmpeg_build/lib" \
 --bindir="$HOME/bin" \
 --extra-version=0ubuntu0.16.04.1 \
 --build-suffix=-ffmpeg \
 --toolchain=hardened \
 --libdir=/usr/lib/x86_64-linux-gnu \
 --incdir=/usr/include/x86_64-linux-gnu \
 --enable-gpl \
 --cc=gcc \
 --cxx=g++ \
 --enable-cross-compile \
 --enable-version3 \
 --enable-shared \
 --disable-stripping \
 --disable-decoder=libopenjpeg \
 --enable-libcdio \
 --enable-libxml2 \
 --enable-openal \
 --enable-ladspa \
 --enable-chromaprint  \
 --enable-avisynth \
 --enable-avresample \
 --enable-libass \
 --enable-libfontconfig \
 --enable-libfreetype \
 --enable-libfribidi \
 --enable-libmp3lame \
 --enable-libtheora \
 --enable-libvorbis \
 --enable-libvpx \
 --enable-libwavpack \
 --extra-cflags=-fPIC \
 --enable-libxvid \
 --enable-libdc1394 \
 --enable-frei0r \
 --enable-libx264 \
 --enable-indev=alsa \
 --enable-outdev=alsa \
 --enable-static \
 --disable-debug \
 --enable-libwebp \
 --enable-libspeex \
 --enable-fontconfig \
 --enable-libopencore-amrnb \
 --enable-libopencore-amrwb \
 --enable-libvo-amrwbenc \
 --enable-gray \ 
 --enable-libgsm \
 --enable-libmysofa \
 --enable-libopenjpeg \
 --enable-libopus \
 --enable-libsoxr \
 --enable-libfribidi \
 --disable-doc \
 --disable-programs \
 --enable-libbluray \
 --enable-libbs2b \
 --enable-libcaca \
 --enable-libdrm \
 --enable-libgme \
 --enable-libopenmpt \
 --enable-libpulse \ 
 --enable-librubberband \
 --enable-libshine \
 --enable-libsnappy \ 
 --enable-libtwolame \
 --enable-libssh 
 


 #--enable-libvo-aacenc 

#RUN PATH="$HOME/bin:$PATH" make
#RUN make install


Overwriting Dockerfile2


In [ ]:
 --enable-libzimg \
     --enable-libzvbi  \
     --enable-libx265 \  
     --enable-opengl \
     --enable-sdl2 \
    
libcdio-dev libcdio-paranoia-dev libqt4-opengl libqt4-opengl-dev

In [ ]:
 --enable-libtwolame \
 --enable-libx265 \    
    libsox-fmt-mp3 libtwolame-dev libtwolame0 libx265-146 libx265-dev x265
libopenal-dev libopenal1

libqt4-opengl libqt4-opengl-dev/


In [ ]:
 --enable-libopenmpt \
 --enable-libpulse \ 
 --enable-librubberband \
 --enable-libshine \
 --enable-libsnappy \ 
 --enable-libssh \   
libopenmpt-dev libopenmpt-modplug-dev libopenmpt-modplug1 libopenmpt0 openmpt123 \
librubberband-dev librubberband2 rubberband-cli rubberband-ladspa libpulse0 libpulse-dev \
libshine-dev libshine3 liquidsoap-plugin-shine shineenc libsnappy-dev libsnappy1v5 \
libssh-4 libssh-dev libavfilter-extra6 libtesseract-dev libtesseract4

In [ ]:
--enable-libbluray \
--enable-libbs2b \
--enable-libcaca \
--enable-libdrm \
--enable-libgme \
# --enable-libgsm \  --enable-libmysofa \
 --enable-libopenmpt \
 --enable-libpulse \ 
 --enable-librubberband \
 --enable-libshine \
 --enable-libsnappy \ 
 --enable-libssh \   
libopenmpt-dev libopenmpt-modplug-dev libopenmpt-modplug1 libopenmpt0 openmpt123 \
librubberband-dev librubberband2 rubberband-cli rubberband-ladspa libpulse0 libpulse-dev \
libshine-dev libshine3 liquidsoap-plugin-shine shineenc libsnappy-dev libsnappy1v5 \
libssh-4 libssh-dev libavfilter-extra6 libtesseract-dev libtesseract4
RUN apt install -y libgme-dev libgme0 libgmenuharness-dev libbluray-bin libbluray-dev libbluray2 \
libgsm-tools libgsm0710-0 libgsm0710-dev libgsm0710mux3 libgsm1 libgsm1-dbg libgsm1-dev libgsmme-dev \
libdrm-amdgpu1 libdrm-common libdrm-dev caca-utils libcaca-dev libcaca0 libcacard-dev libcacard-tools \
libcacard0 libbs2b-dev libbs2b0 libgsm-tools libgsm0710-0 libgsm0710-dev libgsm0710mux3 libgsm1-dbg \
libgsm1-dev libgsmme-dev libmysofa-dev libmysofa-utils libmysofa0 libopencore-amrwb0-dbg \
libopencore-amrnb-dev libopencore-amrnb0 libopencore-amrnb0-dbg libopencore-amrwb-dev libopencore-amrwb0 \

In [ ]:
libopencv-videostab3.2

In [1]:
!ls *.txt

ffmpeg-manual.txt  history2019-1-20.txt  mylist.txt


In [11]:
f = open("history2019-1-20.txt", "r").readlines()
for line in f:
    if "sudo apt install" in line:
        line = line.replace("\n", "")
        line = line.replace("sudo apt install", "") 
        line = line[5:]
        line = line.replace("  ", "")
        line = line.lstrip(' ')
        print line

inetutils-traceroute
python3-numpy python3-matplotlib python3-basemap python3-pygeoip
yasm
frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r
libass-dev libass-devel
libass-dev
libdc1394-22-dev
libmp3lame=3.98.3
libmp3lame==3.98.3
libmp3lame-dev libmp3lame0 libsox-fmt-mp3
libtheora-dev libtheora-bin libtheora0
aptlibvorbis-dev
libvorbis-dev
libvpx-dev
libwavpack-dev libwavpack1
libx264-152 libx264-dev
libx265-146 libx265-dev x265
libx265-146 libx265-dev x265
libxvidcore-dev libxvidcore4
gcc-multilib libc0.1-dev
gcc-multilib
libopenjp2-7-dev libopenjp2-7
libopus-dev
--reinstall librtmp-dev librtmp1
libsoxr-dev libsoxr-lsr0 libsoxr0
speex libspeex-dev
libvo-amrwbenc-dev
libwebp-dev libwebp6
libespeak-dev libespeak1
libav-tools
detox
-y expat libexpat1 libexpat1-dev lua-expat lua-expat-dev node-node-expat xlsx2csv
libexpat1 libexpat1-dev
libexpat1-udeb
libagg-dev
ros-melodic-desktop-full
ros-melodic-desktop-full --fix-missing
ruby-railties
libpng-dev libpnglite-dev libpng-tools 

In [ ]:
yasm frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r libass-dev libass-devel \
libass-dev libdc1394-22-dev libmp3lame-dev libmp3lame0 libsox-fmt-mp3 libtheora-dev \
libtheora-bin libtheora0 libvorbis-dev libvpx-dev libwavpack-dev libwavpack1 \
libx264-152 libx264-dev libx265-146 libx265-dev x265 libx265-146 libx265-dev x265 \
libxvidcore-dev libxvidcore4 libopenjp2-7-dev libopenjp2-7 libopus-dev librtmp-dev librtmp1 \
libsoxr-dev libsoxr-lsr0 libsoxr0 speex libspeex-dev libvo-amrwbenc-dev libwebp-dev libwebp6 \
libespeak-dev libespeak1 libav-tools python3-pyopencl libclc-dev pulseaudio pulseaudio-dev \
youtube-dl winff libavcodec-extra libavcodec-extra57 libvo-aacenc-dev vlc-plugin-access-extra \
meld rubberband-ladspa libiec61883-dev ladspa-sdk libopencv-apps-dev libopencv-apps0d \
libopencv-calib3d-dev libopencv-calib3d3.2 libopencv-contrib-dev libopencv-contrib3.2 \
libopencv-core-dev libopencv-core3.2 libopencv-dev libopencv-features2d-dev \
libopencv-features2d3.2 libopencv-flann-dev libopencv-flann3.2 libopencv-highgui-dev \
libopencv-highgui3.2 libopencv-imgcodecs-dev libopencv-imgcodecs3.2 libopencv-imgproc-dev \
libopencv-imgproc3.2 libopencv-ml-dev libopencv-ml3.2 libopencv-objdetect-dev libopencv-objdetect3.2 libopencv-photo-dev libopencv-photo3.2 libopencv-shape-dev libopencv-shape3.2 libopencv-stitching-dev libopencv-stitching3.2 libopencv-superres-dev libopencv-superres3.2 libopencv-ts-dev libopencv-video-dev libopencv-video3.2 libopencv-videoio-dev libopencv-videoio3.2 libopencv-videostab-dev libopencv-videostab3.2 libopencv-viz-dev libopencv-viz3.2
libchromaprint-dev gnutls-bin libgnutls-openssl27 libgnutls28-dev libgnutls30 libnettle6 nettle-dev \
libhogweed4 locate nano 